## Installing Dependencies

In [120]:
pip install langchain langchain_openai langchain_community streamlit PyPDF2 faiss-cpu langchain-groq

## Setting up the environment

In [121]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()
# Get the API key from the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")


In [122]:
# Load the .env file
load_dotenv()
# Get the API key from the environment variable
groq_api_key = os.getenv("GROQ_API_KEY")


## Imports

In [123]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate 
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_groq import ChatGroq

## Load PDF

In [124]:
pdf = r"C:\Users\91701\Desktop\Healthcare Research Assistant\hrp.pdf"

if pdf is not None:
    pdf_reader = PdfReader(pdf)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

## Chunking

In [125]:

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

## Embedding in VectorDB

In [126]:

chunks = text_splitter.split_text(text)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(chunks, embeddings)

## LLMs

In [127]:
llm2 = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key= groq_api_key[1:57] 
)

In [128]:
from langchain.llms import Ollama
llm1 = Ollama(base_url= "http://localhost:11434", model="tinydolphin")

In [129]:
retriever = vectorstore.as_retriever()

llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0)

## Prompt Tmplate

In [130]:


template = """ you are a helpful healthcare pdf assistant. 
        Given the following pdf, answer the question based on the context.
        If you don't know the answer, just say that you don't know. 
        You may suggest non critical Healthcare issues even if the context is not in the pdf. 
        Never Prescribe medications or become a substitute for doctor.
        Do not make up an answer if you don't know about it.

        Question: {question}
        Context: {context}
        
        Answer:"""

prompt = ChatPromptTemplate.from_template(template)

## Chain

In [138]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm2
    |StrOutputParser()
    
)

## Retrieval

In [139]:

query = "explain the concept of welfare regimes"
answer = rag_chain.invoke(query)

## Response

In [140]:
print(answer)

Based on the provided PDF context, the concept of welfare regimes refers to a typology of welfare states developed by Gøsta Esping-Andersen. This typology aims to identify and categorize different types of welfare states based on their characteristics, such as decommodification, stratification, and the interaction between the market, state, and family.

In the context of healthcare systems, the welfare regime concept is not directly applicable, as the dimensions used to distinguish between welfare regime types do not provide an adequate basis for differentiating between healthcare systems. Instead, a separate and specific typology for healthcare systems is needed.

The welfare regime concept is based on Max Weber's ideal-typical methodology, which aims to identify "ideal-typical" cases that represent distinct types of welfare states. Esping-Andersen's work has been influential in shaping the field of welfare state research, and his typology has been widely used and debated in the liter